In [1]:
import pandas as pd
import pickle

In [2]:
!pip install gensim
#gensim needs to be installed in current environment

In [4]:
from gensim.models import Word2Vec
#no need to tokenize our data cause there are no sentences, just a bunch of illogical nouns




In [5]:
#we create embeddings on the previously created dataset, data.csv
#use word2vec?
#pick appropriate embedding model not whatever we think is popular
#https://medium.com/@mehdi_chebbah/how-to-choose-the-right-embedding-model-for-you-1fc917d14517 -> based on this article, we have to pick static word
#level embedding model->word2vec or GloVe
#using word2vec for now

In [6]:
df = pd.read_csv('../data/structured/groupby_data.csv')
df.head()


,e1,e2,all
0,db00014,"['db00014', 'db00644', 'db00183', 'db06196', '...","['db00014', 'db00014', 'db00644', 'db00183', '..."
1,db00035,"['db00035', 'db00666', 'db00104', 'db00644', '...","['db00035', 'db00035', 'db00666', 'db00104', '..."
2,db00050,"['db00050', 'db00014', 'db00666', 'db06699', '...","['db00050', 'db00050', 'db00014', 'db00666', '..."
3,db00091,['db00091'],"['db00091', 'db00091', 'c0232492', 'c0036690',..."
4,db00093,"['db00093', 'db00781', 'db01282', 'db00035', '...","['db00093', 'db00093', 'db00781', 'db01282', '..."


In [7]:
df.e2 = df.e2.apply(eval)
#e2 is being read as strings, just making them lists

In [8]:
# model1 = Word2Vec([["akhil","akhi"]*70,["surya","parryhotter"]*90,["goparaju","naga","GN","genie"]*50], vector_size=30, window=5, min_count=0, workers=5,epochs=10,sorted_vocab=0)
# model1.wv.index_to_key


In [9]:
# model1.wv.most_similar('genie',topn=5)
#just checking syntax and output formats

In [10]:
model = Word2Vec(df.e2, vector_size=50, window=10, min_count=1, workers=5,epochs=10)
model.wv.most_similar('db00014',topn=5)



[('db01395', 0.866726279258728),
 ('db00123', 0.8640450239181519),
 ('db00328', 0.8638545870780945),
 ('db01047', 0.8620090484619141),
 ('db01559', 0.8619389533996582)]

In [11]:
for j in df.head(5).e1:
    print(j)
    for k,i in enumerate(model.wv.most_similar(j,topn=100)):
        if i[0] in df.iloc[0,1]:
            print(k,i)

db00014
db00035
db00050
db00091
76 ('db00666', 0.3627897799015045)
db00093


In [12]:
#the results are not promising, are embeddings supposed to work for this?


In [13]:
df.e2[df.e2.apply(lambda x: 'db00644' in x)]
# well we cannot certainly say as the same compound may have appeared multiple times and may be the most similar compound from the above has been associated
#more often with current compound

0       [db00014, db00644, db00183, db06196, db00666, ...
1       [db00035, db00666, db00104, db00644, db01282, ...
2           [db00050, db00014, db00666, db06699, db00644]
60                   [db00183, db00666, db00644, db08889]
263     [db00403, db06699, db00014, db00050, db00666, ...
476                           [db00644, db00666, db06196]
1181    [db02638, db00644, db00781, db00626, db00104, ...
1270                          [db06699, db00644, db00666]
Name: e2, dtype: object

In [14]:
df.e2[df.e2.apply(lambda x: 'db00014' in x and 'db00666' in x)]

0      [db00014, db00644, db00183, db06196, db00666, ...
2          [db00050, db00014, db00666, db06699, db00644]
263    [db00403, db06699, db00014, db00050, db00666, ...
Name: e2, dtype: object

In [15]:
df.e2[df.e2.apply(lambda x: 'db00014' in x and 'db00644' in x)]

0      [db00014, db00644, db00183, db06196, db00666, ...
2          [db00050, db00014, db00666, db06699, db00644]
263    [db00403, db06699, db00014, db00050, db00666, ...
Name: e2, dtype: object

In [16]:
#the above two rows sort of support what i thought, the data suggests those two compounds are associated.... this is purely on the basis of
#the number of occurances, i might be gravely wrong... however whoever made the initial knowledge graph dataset might have analysed these compounds at
#molecular level and concluded that they are similar in some way

In [17]:
#i think i should create a dictionary with the side effect as key and known compounds as values...
#our question would be  "what untested compound could have caused this side effect". -> we would get the top 10 similar compounds of each value and then select whichever 
#is the highest and does not occur in the value

In [18]:
side_eff_dict = {}
se = pd.read_csv('../data/structured/side_effects.csv')
se.head(),se.shape

(  side_effect  compound
 0     db00448  c0032584
 1     db00575  c0424024
 2     db00734  c0235309
 3     db06817  c0014724
 4     db00448  c0001824,
 (138944, 2))

In [19]:
for i,j in zip(se.side_effect,se.compound):
    if j in side_eff_dict.keys():
        side_eff_dict[j].append(i)
    else:
        side_eff_dict[j]=[i]


In [20]:
len(side_eff_dict)

5701

In [21]:
#len(se.e2.value_counts()) #got 5701

In [22]:
for i in side_eff_dict.keys():
    side_eff_dict[i]=set(side_eff_dict[i])
    #to have faster indexing

In [25]:
with open('../data/meta/side_effects.pickle', 'wb') as file:
    pickle.dump(side_eff_dict, file)

In [26]:
model.save('../data/meta/compound_embeddings.bin')